In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

---

## NLP 콘텐츠 분류

### 카테고리 전체 불러오기

In [5]:
r = requests.get('https://cloud.google.com/natural-language/docs/categories')

In [6]:
soup = bs(r.text,'lxml')

In [7]:
category_list = list(map(lambda x: x.text, soup.select('table td')))
category_list = [x for x in category_list if x !='']

In [8]:
# 확인
category_list[0:5]

['/Adult',
 '/Hobbies & Leisure',
 '/Arts & Entertainment',
 '/Hobbies & Leisure/Clubs & Organizations',
 '/Arts & Entertainment/Celebrities & Entertainment News']


### 카테고리 대분류 확인하기

In [6]:
large_category = set([x.split('/')[1] for x in category_list])
print('대분류 갯수 :', len(large_category),'개')
print('================')
print('대분류 종류 :', large_category)

대분류 갯수 : 27 개
대분류 종류 : {'Science', 'Law & Government', 'Online Communities', 'Real Estate', 'Finance', 'Travel', 'Games', 'Computers & Electronics', 'Food & Drink', 'Home & Garden', 'Beauty & Fitness', 'Pets & Animals', 'Business & Industrial', 'Sensitive Subjects', 'News', 'Reference', 'Shopping', 'Hobbies & Leisure', 'Books & Literature', 'Arts & Entertainment', 'Internet & Telecom', 'Jobs & Education', 'People & Society', 'Health', 'Sports', 'Autos & Vehicles', 'Adult'}


---

# API 이용하기

## 설치 및 사용 설정

### 참고
- 전체 설치 : https://googlecloudplatform.github.io/google-cloud-python/latest/language/usage.html
- python 예시 : https://cloud.google.com/natural-language/docs/reference/libraries
- 콘텐츠 분류 python 예시 : https://cloud.google.com/natural-language/docs/classifying-text#language-classify-content-python
- 항목 분석 python 예시 : https://cloud.google.com/natural-language/docs/analyzing-entities#language-entities-string-python
- 항목 분석 상세 : https://cloud.google.com/natural-language/docs/reference/rest/v1/Entity
- nlp api 설명(한글) : https://cloud.google.com/natural-language/docs/basics#entity-request
    
### 환경 세팅    
- api를 쓰기 위해선 권한을 설정하고 환경을 세팅해야한다
- 해당 api가 포함될 google cloud project로 이동한 뒤에(본인의 경우 nlp_trial), 새 서비스 계정을 생성한다
    - 역할은 프로젝트 소유자로 설정한다.
    - 이때 받은 json으로 api에 접근하므로 잘 보관한다. 일종의 pem 키인것 같다
- 터미널에서 환경변수 **GOOGLE_APPLICATION_CREDENTIALS** 를 생성하고, 위에서 받은 json 파일의 경로를 넣어준다.
    - ex) export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/service-account-file.json"
    - 이때, 위 환경변수는 현재 열린 shell session에서만 적용되므로, 새로 shell을 열면, 다시 환경변수를 지정해주어야 한다.
    - 따라서 주피터 놋북을 열기 전마다 실행해주어야한다.
        - ***주의*** : 내가 못하는 것인지, 주피터 놋북 안에서는 환경변수가 만들어지지 않는다... ***터미널에서 미리 실행하고 놋북을 열어야한다.***
        
> terminal에 아래 명령어 수정 후 실행
> ```console
> MacBook-Pro:~ username$export GOOGLE_APPLICATION_CREDENTIALS='/home/user/Downloads/service-account-file.json'
> ```

> 실행 후 환경변수가 만들어졌는지 확인. 출력이 되면 만들어진것
> ```console
> MacBook-Pro:~ username$printenv | grep GOOGLE_APPLICATION_CREDENTIALS
> ```

### 가격(월별 사용량, 1000단위당)
- 콘텐츠 분류
    - 0~30,000 단위/월 : 무료
    - 30,001~250,000 단위/월 : 2달러
- 항목 분석
    - 0~5,000 단위/월 : 무료
    - 5,001~1,000,000 단위/월 : 1달러
    
### 기타 주의사항
- 한 번에 분석할 텍스트는 3000자 제한이다.

In [2]:
# 필요 library import
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [18]:
# Instantiates a client
client = language.LanguageServiceClient()

### 감정 분석
- 감정 score는 -1.0(부정적)에서 1.0(긍정적) 사이이며 텍스트의 전반적인 정서 성향을 나타낸다
- magnitude는 주어진 텍스트 내에서 전반적인 감정 강도(긍정적 및 부정적 모두)를 나타내며 0.0부터 +inf까지이다.. score와 달리 magnitude는 정규화되지 않는다. 텍스트 내의 각 감정 표현(긍정적 및 부정적 모두)이 텍스트 magnitude에 반영되며, 따라서 긴 텍스트 블록일수록 값이 더 커진다.
    - 따라서 텍스트의 길이를 통해 magnitude 값을 보정해야한다.

|감정|샘플 값|
|---|---|
|확실히 긍정적*|'score': 0.8, 'magnitude': 3.0|
|확실히 부정적*|'score': -0.6, 'magnitude': 4.0|
|중립적|'score': 0.1, 'magnitude': 0.0|
|혼합|'score': 0.0, 'magnitude': 4.0|

In [26]:
# The text to analyze
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: Hello, world!
Sentiment: 0.30000001192092896, 0.30000001192092896


---

### 콘텐츠 분류

- 위키피디아에서 normal distribution의 intro 부분을 가져와서 시험에 보았다.
- /Science/Mathematics/Statistics 로 완벽하게 분류하는 것을 확인할 수 있따

In [27]:
# Instantiates a client
client = language.LanguageServiceClient()

In [28]:
def classify_text(text):
    """Classifies content categories of the provided text.
    text는 애초에 안전하게 utf-8 형식으로 넣자."""
    
#     client = language.LanguageServiceClient()
#     if isinstance(text, six.binary_type):
#         text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    categories = client.classify_text(document).categories

    for category in categories:
        print(u'=' * 20)
        print(u'{:<16}: {}'.format('name', category.name))
        print(u'{:<16}: {}'.format('confidence', category.confidence))

In [30]:
sample_text = '''In probability theory, the normal (or Gaussian or Gauss or Laplace–Gauss) distribution is a 
                very common continuous probability distribution. Normal distributions are important in statistics 
                and are often used in the natural and social sciences to represent real-valued random variables whose 
                distributions are not known. A random variable with a Gaussian distribution is said to be 
                normally distributed and is called a normal deviate.The normal distribution is useful because of 
                the central limit theorem. In its most general form, under some conditions 
                (which include finite variance), it states that averages of samples of observations of 
                random variables independently drawn from independent distributions converge in distribution to 
                the normal, that is, become normally distributed when the number of observations is sufficiently large.
                Physical quantities that are expected to be the sum of many independent processes 
                (such as measurement errors) often have distributions that are nearly normal. Moreover, many results 
                and methods (such as propagation of uncertainty and least squares parameter fitting) can be derived 
                analytically in explicit form when the relevant variables are normally distributed.The normal 
                distribution is sometimes informally called the bell curve. However, many other distributions are 
                bell-shaped (such as the Cauchy, Student's t, and logistic distributions).'''

# classify_text(sample_text)

name            : /Science/Mathematics/Statistics
confidence      : 0.7799999713897705


---

- 한국어도 되는지 테스트해보자
    - 요새 본인이 빠져있는 사이먼 도미닉(쌈디)의 나무위키 소개글을 넣어보았다.
- api에서 **한국어 지원이 안된다고 반환한다**
    - 이유를 찾았다
    - <a href='https://cloud.google.com/natural-language/docs/languages'>참고</a>: analyzeEntitySentiment 및 classifyText 메소드는 **영어만 지원**합니다.

In [32]:
korean_text = '''前 AOMG의 공동대표, 매력적인 톤과 화려한 랩스킬로 인정받는 래퍼. 이센스와 같이 2000년대 후반 한국 언더그라운드 힙합계를 
                대표하던 루키. 본명은 정기석, 예명은 사이먼 도미닉. 한번 줄이면 사이먼 디 (Simon D), 메이저 시장에서 활동할 땐 언더그라운드 
                시절 리스너들의 애칭이었던 쌈디로 본인을 소개했다. 이름이 짧을수록 기억하기가 쉽기 때문인 듯. E SENS와 함께 힙합그룹 슈프림팀의 
                멤버이며, 박재범과 함께 레이블 AOMG의 수장을 맡고 있었으나 2018년 7월 25일 Me No Jay park 싱글을 통해 대표직을 사임하고 
                소속 아티스트로 남기로했다.'''
# classify_text(korean_text)

---

### 항목 분석(키워드 추출)
- 관련 링크는 **<a href='https://cloud.google.com/natural-language/docs/reference/rest/v1/Entity'>여기</a>**
- 감지된 항목 모음과 해당 항목과 연관된 매개변수(예: 항목 유형, 항목과 전체 텍스트의 관련성, 같은 항목을 참조하는 텍스트 내 위치)를 반환한다.
- 항목은 전체 텍스트와의 관련성을 나타내는 **salience 점수의 내림차순**으로 반환된다.
- 크게 두 가지 카테고리로 나뉜다
    - 고유 명사 : 특정인, 장소
    - 일반 명사 : 자연어 처리에서 nominal 이라고 함.
- 반환되는 key 값 설명
    - **type** : 항목의 유형. 
        - ex)인물/위치/소비재..etc.
        - 여기서 반환되는 값은 콘텐츠 분류에서 반환되는 분류 기준과 일대일 대응이 되지 않는다(아쉽...)
        - type
    - **metadata** : 항목의 지식 저장소에 대한 소스 정보
        - 관련된 위키피디아가 있으면, 해당 url을 반환한다.
        - mid는 Google Knowledge Graph 항목에 해당하는 머신 생성 식별자. 즉, 구글 서치에서 해당 단어에 대한 id 값인듯.
            - 같은 의미 단어라도 언어마다 mid 값이 다를 수도 있는 듯하다. 이를 이용하려면 서로 매칭시켜야 함.
    - **salience** : 전체 텍스트에서 해당 항목의 중요성. 
        - 0.0(안 중요)~1.0(매우 중요)
    - **mentions** : 텍스트 내에서 항목이 언급된 오프셋 위치
        - 정확하게는 이해를 못하고 추상적으로만 이해함...
        - 전체 명사구를 잘라서 lawrence 하나에 집중하고플 때 좋다.
            - 아래 예시에서 Lawrence of Arabia는 실존 인물 T.E Lawrence의 전기 영화이다.
            - 여기서 Lawrence만 잘라서 텍스트에 같이 포함되어있는 T.E Lawrence까지 묶어서 분석할 수 있게 된다.
            - 그니까, Lawrence가 어떤 경우에 언급이 되는지를 다른 T.E. Lawrence의 mention까지 묶어서 확인하면, 더 정보가 많아진다는 의미인듯.
        - PROPER type은 고유명사를 나타내며, COMMON type은 보통 명사를 나타낸다.
            - 아래 예시에서 film biography(전기영화)라는 보통 명사를 반환한 것을 확인할 수 있다.
> ```json
> {"name": "Lawrence of Arabia",
   "type": "WORK_OF_ART",
   "metadata": {"mid": "/m/0bx0l",
                "wikipedia_url": "http://en.wikipedia.org/wiki/Lawrence_of_Arabia_(film)"},
   "salience": 0.75222147,
   "mentions": [{"text": {"content": "Lawrence of Arabia", "beginOffset": 1},
                 "type": "PROPER"
                },
                {"text": {"content": "film biography","beginOffset": 39},
                 "type": "COMMON"
                }]
    }
> ```

In [33]:
# text는 알아서 utf-8로 미리 바꾸고 집어넣어야 한다.
## client는 미리 선언하고 함수를 돌린다.
def entities_text(text):
    """Detects entities in the text."""
#     client = language.LanguageServiceClient()
#     if isinstance(text, six.binary_type):
#         text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects entities in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    entities = client.analyze_entities(document).entities

    # entity types from enums.Entity.Type
    entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
                   'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

    for entity in entities:
        print('=' * 20)
        print(u'{:<16}: {}'.format('name', entity.name))
        print(u'{:<16}: {}'.format('type', entity_type[entity.type]))
        print(u'{:<16}: {}'.format('metadata', entity.metadata))
        print(u'{:<16}: {}'.format('salience', entity.salience))
        print(u'{:<16}: {}'.format('wikipedia_url',
              entity.metadata.get('wikipedia_url', '-')))

In [16]:
# 영어, normal distribution
entities_text(sample_text)

name            : probability distribution
type            : OTHER
metadata        : {}
salience        : 0.39728057384490967
wikipedia_url   : -
name            : distribution
type            : OTHER
metadata        : {}
salience        : 0.07848833501338959
wikipedia_url   : -
name            : Gaussian
type            : PERSON
metadata        : {}
salience        : 0.06503112614154816
wikipedia_url   : -
name            : probability theory
type            : OTHER
metadata        : {}
salience        : 0.05950438976287842
wikipedia_url   : -
name            : distributions
type            : OTHER
metadata        : {}
salience        : 0.04490067437291145
wikipedia_url   : -
name            : Gauss
type            : PERSON
metadata        : {'mid': '/m/01t_z', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss'}
salience        : 0.04394334927201271
wikipedia_url   : https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss
name            : Laplace
type            : LOCAT

In [17]:
# 한국어, 쌈디
entities_text(korean_text)

name            : AOMG
type            : ORGANIZATION
metadata        : {'mid': '/m/011q0tgb', 'wikipedia_url': 'https://en.wikipedia.org/wiki/AOMG'}
salience        : 0.05813010782003403
wikipedia_url   : https://en.wikipedia.org/wiki/AOMG
name            : 한국 언더그라운드 힙합계
type            : ORGANIZATION
metadata        : {}
salience        : 0.0540737621486187
wikipedia_url   : -
name            : 이센스
type            : OTHER
metadata        : {'mid': '/g/121p193b', 'wikipedia_url': 'https://en.wikipedia.org/wiki/E_Sens'}
salience        : 0.04800480604171753
wikipedia_url   : https://en.wikipedia.org/wiki/E_Sens
name            : 박재범
type            : PERSON
metadata        : {'mid': '/m/0bh8fjc', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Jay_Park'}
salience        : 0.04598141089081764
wikipedia_url   : https://en.wikipedia.org/wiki/Jay_Park
name            : 슈프림
type            : PERSON
metadata        : {}
salience        : 0.04598141089081764
wikipedia_url   : -
name          

#### 핵잘됨.